In [27]:
import torch
import torch.nn as nn
from torch.nn import functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 10000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

cuda


In [28]:
with open('wizard_of_oz.txt', 'r', encoding = 'utf-8') as f:
    text = f.read()
chars = sorted(set(text))
#print(chars)
vocab_size = len(chars)

In [29]:
string_to_int = { ch:i for i, ch in enumerate(chars) }
int_to_string = { i:ch for i, ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: "".join([int_to_string[i] for i in l])

#encoding the data from the wizard of oz
data = torch.tensor(encode(text), dtype = torch.long)

In [30]:
#train on 80% then 20% do this to not just generate text 1:1 as the given data but to speak like it
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[69, 65, 58, 54, 72, 74, 71, 58],
        [66, 58,  1, 68, 67, 58,  1, 54],
        [ 1, 54,  1, 61, 74, 66, 55, 74],
        [66,  1, 73, 61, 58,  1, 71, 58]], device='cuda:0')
targets:
tensor([[65, 58, 54, 72, 74, 71, 58, 11],
        [58,  1, 68, 67, 58,  1, 54, 55],
        [54,  1, 61, 74, 66, 55, 74, 60],
        [ 1, 73, 61, 58,  1, 71, 58, 75]], device='cuda:0')


In [31]:


x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print("when target is ",context,"target is", target)

when target is  tensor([80]) target is tensor(1)
when target is  tensor([80,  1]) target is tensor(1)
when target is  tensor([80,  1,  1]) target is tensor(28)
when target is  tensor([80,  1,  1, 28]) target is tensor(39)
when target is  tensor([80,  1,  1, 28, 39]) target is tensor(42)
when target is  tensor([80,  1,  1, 28, 39, 42]) target is tensor(39)
when target is  tensor([80,  1,  1, 28, 39, 42, 39]) target is tensor(44)
when target is  tensor([80,  1,  1, 28, 39, 42, 39, 44]) target is tensor(32)


In [32]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [33]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss

    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


Jv'"2r"dUA;rtXWyRrx 0RgoKMQIn5El!e!
WIfi&sX";KCypFbwe)7AVuEfXEbGtG[BlbQHlBvc4i)yjBnY*sR8Z9TjU0 4;],
.2(M85L9RYZ*q
m]q.2Ec(_OQPg0U(qm6nP75*N15!
53Hlprq4)k﻿A
MLTDaZy&BTy7qB1SEdJMJXd:!yG9LJ_﻿j6O;bQY6BdyRbOsR﻿5-1KuOBrDM*nO9!LKes[QGiB1v4:)FVkHFn"!J_f(.c_LX"r1bP
yEcxTKvXUMU0VMACe5HKYip.RY0srY.!
.-Kg8"M'e)Qdys2iyX2159UAVk_NCZwHZIn?qBPi4LgZXEA*.f&)2iOsJS"i
J8iLvtzlK'l(t1wqg8 SRFVkM86R[*8Rd2(.8﻿i
"te&4(zFG&nZqBL"RhEDz60u1M&ueuO*C*U&r!5]v!;WMk]*LznmRGF(9vyERZypOfX4;]L"wnY)*B_zFxvqagcvuWauD]n1VHc*jUPiN.HwQ


In [34]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.924, val loss: 4.895
step: 250, train loss: 4.838, val loss: 4.812
step: 500, train loss: 4.775, val loss: 4.776
step: 750, train loss: 4.711, val loss: 4.699
step: 1000, train loss: 4.661, val loss: 4.647
step: 1250, train loss: 4.597, val loss: 4.586
step: 1500, train loss: 4.541, val loss: 4.549
step: 1750, train loss: 4.493, val loss: 4.501
step: 2000, train loss: 4.431, val loss: 4.440
step: 2250, train loss: 4.381, val loss: 4.384
step: 2500, train loss: 4.301, val loss: 4.334
step: 2750, train loss: 4.263, val loss: 4.263
step: 3000, train loss: 4.207, val loss: 4.215
step: 3250, train loss: 4.162, val loss: 4.159
step: 3500, train loss: 4.127, val loss: 4.106
step: 3750, train loss: 4.054, val loss: 4.075
step: 4000, train loss: 4.039, val loss: 4.029
step: 4250, train loss: 3.982, val loss: 3.987
step: 4500, train loss: 3.934, val loss: 3.940
step: 4750, train loss: 3.911, val loss: 3.898
step: 5000, train loss: 3.850, val loss: 3.863
step: 5250, train l

In [26]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


75W;NRIxPEpqewcXSO9!,VwU2MmoIAaXo Emj4p﻿:7J]r[4.p6kA4'ynJ﻿Qsb*V
wjF3,3fTgFKx;mid[l?]aPcXjhq -FabERJ7UZAoKt9b9ds*L8b_-zVw!L2ond"WPp3zAf.:76bhk1C--KvfZq)vf)mjAk"oamo.&0CvmwvL[&fNRwWPsGMz50URzj)t,PEwVFD
HssP-nRJSRJRYV;Smw"nYNKn["cj"Ok]mi;YYYr﻿Z:7UWr,MjYmm!eB7Ug﻿_YVv.8;[b:7QRJAoADNK)W! -7I)A8XBVMZ-uHQRt1dx_[5UFe5Gul2-vKN[r﻿7SC8
ZSFKMhvszEw1]Ok
h'*vYY*rCw(K mIsbo0zlS6!)Z:)GT](f 5'*UR?)WQ?yiu&He 768cc"6 ]3:7YD,rJAwvT4YYLohb6_KYuMCgU﻿!eBkCfETCfwS6D7gDDVweWNPHs8e0&tRYVOHFE.t[*]kDN2Co_nm!Fq1MC8D;rFKRPAMG
